In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_text")
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_mluvci")
).drop_nulls(
    subset=['datum']
)

In [4]:
df.sample(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,list[str],list[str]
"""Předseda FS A. Indra""","""1971fs_slsn_stenprot_020schuz_s020013.htm""",null,"""Děkuji poslanci Auerspergovi. Soudružky a soudruzi poslanci, pokračovat v jednání budeme přesně v 16.15 hod. Jako první po přestávce vystoupí místopře…",2,"""fs""",1971,20,13,1975-11-11,"""FS ČSSR 1971-1976""","""20. schůze""",true,true,"[""jednání přerušeno v 15.45 hod."", ""jednání opět zahájeno v 16.15 hod.""]",[]
"""Znovu bude hlasovať Snemovňa národov. Pýtam sa""","""1990fs_slsn_stenprot_009schuz_s009065.htm""",null,"""Kto je za takto predložený návrh zákona, nech zdvihne ruku a stlačí tlačítko! (Hlasuje se.) (Devadesát sedm.) Devťdesiatsedem. Ďakujem. Kto je proti? …",11,"""fs""",1990,9,65,1990-12-05,"""Federální shromáždění ČSFR, 1990 – 1992 Společné schůze SL a SN""","""9. schůze""",true,true,"[""hlasuje se."", ""devadesát sedm."", … ""potlesk.""]",[]
null,"""1998ps_stenprot_039schuz_s039273.htm""",null,"""(17.50 hodin) (pokračuje Lachnit) Nicméně to není důležité. Činnost příslušných orgánů všech úrovní, tj. v úrovni místní, okresní, krajské a ústřední,…",1,"""ps""",1998,39,273,2001-10-25,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""39. schůze""",true,true,"[""17.50 hodin"", ""pokračuje lachnit""]",null
"""Hlasování číslo 117""","""2021ps_stenprot_045schuz_s045136.htm""",null,""", přihlášeno 175 poslanců a poslankyň, pro 79, proti 90. Návrh byl zamítnut. Kolegové a kolegyně, prosím o klid!""",45,"""ps""",2021,45,136,2022-11-30,"""PČR, PS 2021-...""","""45. schůze""",true,true,[],[]
"""Místopředseda PSP Vojtěch Filip""","""2013ps_stenprot_056schuz_s056085.htm""",null,"""Děkuji, pane předsedo vlády. Nyní faktická poznámka pana poslance Jana Klána, poté pana poslance Marka Černocha a poté pan kolega Kováčik a další čtyř…",8,"""ps""",2013,56,85,2017-04-06,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""56. schůze""",true,true,[],[]


In [5]:
len(df)

795227

In [27]:
def zavorka(retezec, doslova=True, frejm=df):
    return frejm.explode("poznamky_text").explode("poznamky_mluvci").filter(
            pl.col('poznamky_text').str.contains(retezec, literal=doslova) 
        |
            pl.col('poznamky_mluvci').str.contains(retezec, literal=doslova) 
    ).unique(
        subset=['datum','cast','poradi']
).sort(by='datum').group_by_dynamic(
    index_column='datum',every='1y').agg(pl.col('text').unique().len()
).join(
    frejm.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()), 
    on='datum', how='left'
).with_columns(
    (pl.col('text') / pl.col('text_right')).alias('podil')
).with_columns(
        pl.lit(retezec).alias('co')
)

In [7]:
'; '.join(df.explode("poznamky_text").group_by(
    "poznamky_text"
).len(
).sort(
    by='len',
    descending=True
).head(500).filter(pl.col('poznamky_text').is_not_null()).select(pl.col('poznamky_text')).to_series().to_list())

'děje se.; potlesk.; zvoní; výborně!; nikdo.; souhlas.; hlasuje se.; výkřiky.; nebyly.; tak jest!; výborně! potlesk.; výkřiky komunistických poslanců.; čte; ano.; ?; 1; 2; námitky nebyly.; slyšte!; hluk.; veselost.; nesouhlasné.; nesouhlas.; hluk v sále.; potlesk zprava.; výborně! - potlesk.; souhlas na levici.; potlesk komunistických poslanců.; ministr: nesouhlas.; 3; námitek nebylo.; potlesk; slibuji.; smích v sále.; ne.; nesrozumitelné; pokračuje; předsedající: čas.; německy; 4; potlesk poslanců čssd.; ministr: souhlas.; potlesk poslanců ods.; potlesk na levici.; potlesk zleva.; upozornění na čas; potlesk poslanců ksčm.; výkřiky na levici.; nikdo se nehlásil.; potlesk v části sálu.; pokračuje langer; nebyla.; smích.; tak je!; pokračuje filip; souhlasné.; výkřiky sen. mikulíčka.; děje se; potlesk části poslanců.; různé výkřiky.; předsedající upozorňuje na čas.; pokračuje zaorálek; 14.40 hodin; 14.50 hodin; deje sa.; 15.10 hodin; 15.20 hodin; 15.00 hodin; na neurčito; 15.40 hodin; upo

In [8]:
alt.Chart(
    zavorka("potlesk").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [9]:
alt.Chart(
    zavorka("smích").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [10]:
alt.Chart(
    zavorka("výkřiky").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [11]:
df.filter(pl.col("text").str.contains("[Bb]učení"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,list[str],list[str]
"""Ministr zdravotnictví ČR David Rath""","""2002ps_stenprot_055schuz_s055136.htm""",null,"""Děkuji. Je to myslím docela zase signifikantní a typické, když názor vyslovený těmi, kterých se to bytostně týká, tedy lékaři zaměstnanci a zdravotním…",15,"""ps""",2002,55,136,2006-04-21,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true,[],[]
"""Poslanec David Rath""","""2006ps_stenprot_027schuz_s027049.htm""",null,"""Dámy a pánové, pane předsedající, já bych prosil, abyste poučil o slušném chování poslance ODS, aby zde nebučeli. Nejsou, jak jsou zvyklí, v chlívě. Z…",3,"""ps""",2006,27,49,2008-01-30,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""27. schůze""",true,true,[],[]
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_040schuz_s040102.htm""",null,"""Děkuji za slovo. Protože nemáme tu britskou tradici, tak tady neprobíhalo během předchozích projevů bučení a podobně, místo toho všeobecný nezájem pří…",7,"""ps""",2017,40,102,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,"[""potlesk poslanců pirátů.""]",[]
"""Předsedkyně PSP Markéta Pekarová Adamová""","""2021ps_stenprot_005schuz_s005019.htm""",null,"""Vzhledem k tomu, že program schůze nebyl schválen a nikdo další se s přednostním právem nehlásí, tak já tuto 5. schůzi Poslanecké sněmovny končím. (Po…",2,"""ps""",2021,5,19,2021-12-15,"""PČR, PS 2021-...""","""5. schůze""",true,true,"[""poslanci tlučou do lavic, ozývá se bučení a volání."", ""schůze skončila ve 12.14 hodin.""]",[]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""",null,"""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]
"""Poslanec Jakub Michálek""","""2021ps_stenprot_055schuz_s055007.htm""",null,"""Děkuji za slovo. Vážená paní předsedkyně, vážené kolegyně, vážení kolegové, pojďme prosím nechat v rozpravě vystoupit i ministry, například tady pana …",9,"""ps""",2021,55,7,2023-02-28,"""PČR, PS 2021-...""","""55. schůze""",true,true,"[""nesouhlas a bučení poslanců ano.""]",[]


In [12]:
alt.Chart(
    zavorka("bučen").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
df.filter(pl.col("text").str.contains("(Buče",literal=True))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,list[str],list[str]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""",null,"""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]


In [14]:
alt.Chart(
    zavorka("posmě[cš]",doslova=False).to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [15]:
df.filter(
    pl.col("datum").dt.year().is_between(1940,1970)
).filter(
    pl.col("text").str.contains("(Smích",literal=True)
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,list[str],list[str]
null,"""1946uns_stenprot_056schuz_s056018.htm""",null,"""Chci zase říci, že naši ministři na tom vinni nejsou. Ministr dr Stránský, ministr dr Dolanský, ministr Nosek jsou ochotni tyto věci udělat, ale oni t…",1,"""uns""",1946,56,18,1947-06-10,"""Ústavodárné NS RČS 1946-1948""","""56. schůze""",true,true,"[""hlas: a sedí na ministerstvu dál!"", ""smích."", … ""zvoní""]",null
"""Posl. Harus""","""1960ns_stenprot_004schuz_s004023.htm""",null,"""Vážení soudruzi a soudružky poslanci, chci se v první řadě omluvit, neboť jak jste byli svědky, přihlásil jsem se ke slovu teprve nyní, při projednává…",3,"""ns""",1960,4,23,1960-11-17,"""NS RČS 1960-1964""","""4. schůze""",true,true,"[""hlasy: kým?"", ""oživení v sále"", … ""smích""]",[]
"""Posl. Fogel""","""1960ns_stenprot_010schuz_s010009.htm""",null,"""Súdružky a súdruhovia poslanci! Vládny návrh zákona o disciplinárnej odpovednosti sudcov z povolania, ktorý dnes prejednávame, je jedným z mnohých opa…",6,"""ns""",1960,10,9,1961-11-29,"""NS RČS 1960-1964""","""10. schůze""",true,true,"[""smích.""]",[]
"""Posl. Borůvka""","""1960ns_stenprot_010schuz_s010018.htm""",null,"""Soudružky a soudruzi poslanci! Plně potvrzuji pravdivost zprávy soudruha Šimůnka o vážné situaci v zemědělské výrobě, a to jak z hlediska vlastních zk…",2,"""ns""",1960,10,18,1961-11-30,"""NS RČS 1960-1964""","""10. schůze""",true,true,"[""potlesk."", ""potlesk."", … ""hlas z pléna: vždyť je něco podobného v uhříněvsi.""]",[]
"""Zpravodaj posl. Borůvka""","""1960ns_stenprot_013schuz_s013007.htm""",null,"""Vážené Národní shromáždění, soudruzi a soudružky poslanci! Naše sociální a zdravotní zabezpečení dělníků a ostatních zaměstnanců je bezesporu nejlepší…",4,"""ns""",1960,13,7,1962-03-29,"""NS RČS 1960-1964""","""13. schůze""",true,true,"[""smích.""]",[]
null,"""1960ns_stenprot_023schuz_s023016.htm""",null,"""A i já využívám právě teď příležitosti, abych stejně naléhavě připomněl slib, který nám byl dán v pátek 24. ledna t. r. náměstkem předsedy SPK s. Dvoř…",1,"""ns""",1960,23,16,1964-01-30,"""NS RČS 1960-1964""","""23. schůze""",true,true,"[""potlesk."", ""smích."", ""potlesk.""]",null
"""Ministr s. Lomský""","""1964ns_stenprot_008schuz_s008038.htm""",null,"""Vážené soudružky a soudruzi poslanci, s. posl. inž. Gudrich mne interpeloval v otázce uvolnění továrního objektu v jeho volebním obvodě pro rozvoj výr…",2,"""ns""",1964,8,38,1965-11-12,"""NS RČS 1964-1968""","""8. schůze""",true,true,"[""smích v sále"", ""smích""]",[]
"""Post. Harus""","""1964ns_stenprot_008schuz_s008038.htm""",null,"""K tomu, že jsem poprosil soudruha předsedu o udělení slova po soudruhu ministrovi mě vyprovokoval ministr sám. Právě o této věci jsem s ním včera hovo…",6,"""ns""",1964,8,38,1965-11-12,"""NS RČS 1964-1968""","""8. schůze""",true,true,"[""smích v sále"", ""smích v sále"", … ""smích, potlesk.""]",[]
"""Posl. Kopecká""","""1964ns_stenprot_011schuz_s011033.htm""",null,"""Soudružky a soudruzi, nebojte se, mám jen dotaz. (Smích.) Jako poslanec bych samozřejmě neměla přicházet s požadavkem na pracovní síly a na koruny, al…",2,"""ns""",1964,11,33,1966-06-30,"""NS RČS 1964-1968""","""11. schůze""",true,true,"[""smích."", ""potlesk.""]",[]


In [16]:
alt.Chart(
    pl.concat([zavorka("smích"), zavorka("potlesk"), zavorka("křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [17]:
alt.Chart(
    pl.concat([zavorka("potlesk zleva"), zavorka("potlesk zprava")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [50]:
def obrat(cislo):
    if cislo >= 1:
        return cislo
    if cislo < 1:
        return -(1/cislo)

In [54]:
obrat(0.1)

-10.0

In [86]:
pravolevo = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['datum','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['datum','zprava'])
    ), on="datum", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
).filter(
    pl.col('datum').dt.year() >= 1993
)
pravolevo

datum,zleva,datum_right,zprava,pravo-levo
date,f64,date,f64,f32
1995-01-01,0.000345,1995-01-01,0.000115,-3.0
1996-01-01,0.000328,1996-01-01,0.000657,2.0
1997-01-01,0.00107,1997-01-01,0.000252,-4.25
1998-01-01,0.001139,1998-01-01,0.000526,-2.166667
1999-01-01,0.001842,1999-01-01,0.002201,1.194444
2000-01-01,0.000975,2000-01-01,0.001108,1.136364
2001-01-01,0.000311,2001-01-01,0.000777,2.5
2002-01-01,0.000388,2002-01-01,0.001293,3.333333
2003-01-01,0.000481,2003-01-01,0.003895,8.1


## Po 13 letech se víc tleská zleva!

In [88]:
alt.Chart(
    pravolevo.to_pandas(),
    title=["„Potlesk zprava“ (> 0) a „potlesk zleva“ (< 0)","ve stenozáznamech PS PSP ČR"]
).mark_bar().encode(alt.Y('datum:T'),alt.X('pravo-levo:Q',axis=alt.Axis(title="<– víc se tleská zleva | víc se tleská zprava ->")))

alt.Chart(...)

In [18]:
alt.Chart(
    pl.concat([zavorka("pobavení"), zavorka("veselost"), zavorka("veselí")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [19]:
alt.Chart(
    pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_24268\4207413198.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
C:\Users\micha\AppData\Local\Temp\ipykernel_24268\4207413198.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [20]:
alt.Chart(
    pl.concat([zavorka("(Smích", doslova=True), zavorka("(Veselí", doslova=True), zavorka("(Pobavení", doslova=True)]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [21]:
alt.Chart(
    df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('text:Q', axis=alt.Axis(title=None)),
)

alt.Chart(...)

In [22]:
df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len())

datum,text
date,u32
1918-01-01,614
1919-01-01,3480
1920-01-01,4810
1921-01-01,6618
1922-01-01,6295
1923-01-01,4555
1924-01-01,5045
1925-01-01,3927
1926-01-01,5898
